In [3]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix

<h1> Function and Class Definitions

In [4]:
# utility sigmoid function
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [14]:
class NeuralNet(object):
    
    def __init__(self,hidden_layer_sizes):
        self.hidden_layer_sizes = hidden_layer_sizes
        self.weights = []
        self.n_layers = len(self.hidden_layer_sizes)
        
    #initialize weights and biases
    def w_init(self,X, set_seed=0):
        
        w_init=[]
        np.random.seed(set_seed)
        
        for i in range(self.n_layers+1):
            n_neurons = self.hidden_layer_sizes[i] if i <  self.n_layers else 1 #will need to change if multiple classes
            n_feat = X.shape[1]+1 if i==0 else self.hidden_layer_sizes[i-1]+1
            w_init.append(np.array(0.1*np.random.rand(n_neurons,n_feat)))
        self.weights = w_init
    
    #feedforward
    def feed_forward(self,X):
        
        bias_units=np.ones(X.shape[0])
        a = [X]
                
        for i in range( self.n_layers+1):
            w = self.weights[i]
            a[i]=np.column_stack((bias_units,a[i]))
            z = np.inner(a[i],w)

            a.append(sigmoid(z))
        
        return a

    #backpropagation
    def backprop(self, X, Y):
        a = self.feed_forward(X)
        n_data = X.shape[0]

        D = [np.zeros_like(w) for w in self.weights]

        for i, (x, y) in enumerate(zip(X, Y)):
            delta = [[] for ind in range( self.n_layers + 2)]
            grad_J = [[] for ind in range( self.n_layers+1)]


            delta[-1] = a[-1][i]-Y[i]
            grad_J[-1] = delta[-1] * a[-2][i]
            D[-1] += grad_J[-1]/n_data
            
            for l in reversed(range(1, self.n_layers + 1)):
                g_prime = a[l][i] * (1-a[l][i])
                g_prime = g_prime.reshape(-1,1)
                
              
                #this is here to skip over the bias units for layers other than the last one (which doesn't have a bias unit since it is the output)
                if l ==  self.n_layers:
                    delta[l] = np.dot(self.weights[l].transpose(), delta[l+1].reshape(-1,1)) * g_prime
                else:
                    delta[l] = np.dot(self.weights[l].transpose(), delta[l+1][1:]) * g_prime

                grad_J[l-1] = delta[l][1:] * a[l-1][i]
                D[l-1] += grad_J[l-1]/n_data

        return D
    
    #train NN on data with learning rate lr
    def train(self, X, Y, lr):
        
        #this could be improved by using an error tolerance rather than fixed number of iterations
        for _ in range(1000):
            D = self.backprop(X, Y)
            delta_w = [-1.0*lr*d for d in D]
            self.weights = [w + d for w,d in zip(self.weights, delta_w)]
        
    #numeric gradient check based on one sided finite difference to check backprop
    def grad_check(self,X,Y):
        X_temp=X
        Y_temp=Y.reshape(-1,1)

        numeric_grad=[np.zeros_like(w) for w in self.weights]

        eps=0.001

        for i in range(len(self.weights)):
            for j in range(self.weights[i].shape[0]):
                for k in range(self.weights[i].shape[1]):
                    H0=self.feed_forward(X_temp)[-1]
                    C0=nn_cost(H0,Y_temp)
                    
                    self.weights[i][j,k]+=eps
                    
                    H1=self.feed_forward(X_temp)[-1]
                    C1=nn_cost(H1,Y_temp)
                    
                    self.weights[i][j,k]-=eps

                    numeric_grad[i][j,k]=(C1-C0)/eps
        return numeric_grad
    
    #NN cost function
    def nn_cost(self, H, Y):
        J=-1/H.shape[0]*np.sum(Y*np.log(H)+(1-Y)*np.log(1-H))
        return J

<h1> Data Import

In [6]:
iris=pd.read_csv('Iris.csv')

iris['Species']=iris['Species'].astype('category')

X=iris.loc[:,['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']]
X=np.array(X)

iris['virginica_vall']=iris['Species']=='Iris-virginica'
iris['virginica_vall']=iris['virginica_vall'].astype(int)
Y=iris['virginica_vall']
Y=np.array(Y)

<h1> Model Performance on Iris Dataset

In [9]:
#test against Iris dataset
test=NeuralNet(np.array([2]))
test.w_init(X)
test.train(X,Y,0.1)
confusion_matrix([round(t[0]) for t in test.feed_forward(X)[-1]],Y)

array([[97,  0],
       [ 3, 50]], dtype=int64)

In [10]:
#logistic regression for comparison
log_reg=LogisticRegression(C=1E10, solver='lbfgs')
pred=log_reg.fit(X,Y).predict(X)
confusion_matrix(pred,Y)

array([[99,  1],
       [ 1, 49]], dtype=int64)

In [11]:
#skikit classifier for comparison. NOTE: performance depends on initial conditions
mlp= MLPClassifier(hidden_layer_sizes=(2),solver= 'sgd', activation='relu', alpha=0, batch_size='auto', max_iter=10000)
mlp_pred=mlp.fit(X,Y).predict(X)
confusion_matrix(mlp_pred,Y)

array([[97,  0],
       [ 3, 50]], dtype=int64)

In [12]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(3, 3))#, random_state=1)
Y_all=iris['Species']
Y_all=np.array(Y_all)
confusion_matrix(clf.fit(X,Y_all).predict(X),Y_all)

array([[50,  0,  0],
       [ 0, 49,  1],
       [ 0,  1, 49]], dtype=int64)